In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import numpy as np
import xgboost

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('train.csv', header = None)
df = df.set_index(df.columns[0])
df = df.iloc[0: , :]
df.head()

,1,2,3,4,5,6,7,8,9,10,...,952,953,954,955,956,957,958,959,960,961
0,,,,,,,,,,,,,,,,,,,,,
1,-1.8226,8.51070,-0.53551,-2.03440,2.2282,2.04870,-5.81380,-4.4169,2.3129,0.808290,...,-5.74060,12.5650,41.23100,-6.77570,1.72780,15.9120,23.583000,3.9158,35.7250,1
2,1.2816,-0.49410,-0.19667,1.05510,-1.9280,1.07120,-0.19667,-4.6669,3.8230,-3.245900,...,-20.50300,47.7540,2.88560,-6.86660,6.29350,-4.8789,-1.300100,-1.4275,-3.4192,1
3,-2.0304,-0.27552,-0.58236,-5.95250,7.3550,2.16640,-4.71510,17.1580,15.5120,-0.822280,...,-10.71800,-4.5436,1.38680,-5.32100,-6.08350,-1.3021,-10.569000,-4.2189,-1.6538,1
4,-1.0026,0.41320,0.62058,-0.34226,-0.2027,-0.92619,0.61123,-2.0824,-3.5442,-0.043659,...,-2.90100,4.5538,-0.77261,4.25520,0.27911,4.7744,-5.695200,-18.7410,58.3810,1
5,1.2754,25.84200,5.80000,2.89000,3.1957,4.45040,5.99760,-6.4947,2.9189,3.202700,...,-0.97276,3.8810,7.13330,0.70157,-1.69570,1.2263,-0.093752,-1.7140,2.5131,1


In [3]:
# training data
df = df.sample(frac = 1)
X_train = df.iloc[:,:-1]
y_train = df.iloc[:,-1]

print("X_train shape:", X_train.shape, "| y_train shape:", y_train.shape)


X_train shape: (9388, 960) | y_train shape: (9388,)


In [4]:
# test data
X_test= pd.read_csv('test.csv', header=None)
X_test = X_test.set_index(X_test.columns[0])
X_test.head()
print("X_test shape:", X_test.shape)

X_test shape: (2959, 960)


In [5]:
# minmaxscale data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority', random_state=42)

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(X_train, y_train)

In [7]:
print("x_smote shape:", x_smote.shape, "| y_smote shape:", y_smote.shape)

x_smote shape: (9954, 960) | y_smote shape: (9954,)


In [8]:
rows, cols = x_smote.shape
x_smote_tmp = np.random.rand(rows, cols + 1)
x_smote_tmp[:,:-1] = x_smote
x_smote_tmp[:, -1] = y_smote

print(type(x_smote_tmp))
print(x_smote_tmp.shape)
# print(x_smote_tmp[1,:])

<class 'numpy.ndarray'>
(9954, 961)


In [9]:
type(X_test)
pd.DataFrame(x_smote_tmp).to_csv("train_smote.csv", header=None)
pd.DataFrame(X_test).to_csv("test_smote.csv", header=None)
# pd.DataFrame(X_test).to_csv("test_smote.csv", header=None, index=None)

In [10]:
print(X_test.shape)

(2959, 960)
